In [106]:
import pandas as pd
from Main import settings, features, pf_set
import time
import pickle
from pandas.tseries.offsets import DateOffset
import Prepare_Data
import data_run_files
import return_prediction_functions
from return_prediction_functions import rff
import numpy as np
from pandas.tseries.offsets import MonthEnd
from sklearn.linear_model import Ridge
import General_Functions
import pickle
import Estimate_Covariance_Matrix
import os
import prepare_portfolio_data
import portfolio_choice_functions
import scipy.linalg

start_date = pd.to_datetime('1952-12-31')

#Relevante paths
output_path = "./data_fifty/"
pickle_file_path = "./data_fifty/model_1.pkl"
wealth_path = "./data_fifty/wealth.csv"
barra_file_path = "./data_fifty/barra_cov.pkl"
output_path_market_returns = "data_fifty/market_returns_test.csv"

risk_free_path = "./data_fifty/risk_free_test.csv" 
output_data_ret_csv = "./data_fifty/data_ret.csv"  
output_data_ret_ld1_csv = "./data_fifty/data_ret_ld1.csv"

file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"

risk_free = data_run_files.process_risk_free_rate(risk_free_path, start_date)
daily_path = "./data_fifty/daily.csv"
chars_path = "./data_fifty/chars_behandlet.parquet"


Filtrering udført. Antal rækker efter filtrering: 841
Fil gemt som ./data_test/risk_free_test.csv


In [107]:
wealth = pd.read_csv(wealth_path)
wealth['eom'] = pd.to_datetime(wealth['eom'])
data_ret = pd.read_csv(output_data_ret_csv)
data_ret_ld1 = pd.read_csv(output_data_ret_ld1_csv)
data_ret['eom'] = pd.to_datetime(data_ret['eom'])
data_ret_ld1['eom'] = pd.to_datetime(data_ret_ld1['eom'])
data_ret_ld1['eom_ret'] = pd.to_datetime(data_ret_ld1['eom_ret'])
daily = pd.read_csv(daily_path, parse_dates=["date", "eom"])
chars = pd.read_parquet(chars_path)

with open(pickle_file_path, "rb") as file:
    model_1 = pickle.load(file)


with open(barra_file_path, "rb") as file:
    barra_cov = pickle.load(file)

chars = prepare_portfolio_data.add_return_predictions(chars,settings, output_path)

lambda_list = prepare_portfolio_data.create_lambda_list(chars)

first_cov_date, hp_years, start_oos = prepare_portfolio_data.define_important_dates(barra_cov, settings)
date_ranges = prepare_portfolio_data.create_date_ranges(settings, first_cov_date, start_oos, hp_years)
dates_m1 = date_ranges["dates_m1"]
dates_m2 = date_ranges["dates_m2"]
dates_oos = date_ranges["dates_oos"]
dates_hp = date_ranges["dates_hp"]
#Disse er alle korrekte

In [108]:
def w_fun(data, wealth, dates, w_opt):
    """
    Opdateret w_fun, der sikrer, at 'eom' er en kolonne (ikke en DatetimeIndex),
    og som håndterer både skalar- og listeinput for dates.
    
    Parametre:
      - data: DataFrame med mindst kolonnerne 'id', 'eom', 'tr_ld1'
      - wealth: DataFrame med mindst kolonnerne 'eom', 'mu_ld1'
      - dates: En liste eller en enkelt dato (skalar) – konverteres til pd.Timestamp
      - w_opt: DataFrame med porteføljeoptimale weights (skal indeholde 'id', 'eom' og 'w')
    """
    # Hvis 'dates' er en enkelt dato, pak den ind i en liste
    if not hasattr(dates, '__iter__') or isinstance(dates, str):
        dates = [dates]
        
    # Konverter dates til pd.Timestamp og filtrer ugyldige værdier
    valid_dates = []
    for d in dates:
        try:
            valid_dates.append(pd.to_datetime(d))
        except Exception as e:
            print(f"Skipping invalid date: {d} ({e})")
    dates = valid_dates

    # Opret et DataFrame med 'eom' og 'eom_prev'
    dates_df = pd.DataFrame({'eom': dates})
    dates_df['eom_prev'] = dates_df['eom'].shift(1)
    
    # Udregn initiale vægte via initial_weights_new
    # Her antages det, at funktionen returnerer en DataFrame med 'id', 'eom' (som kolonne) og 'w'
    w = General_Functions.initial_weights_new(data, w_type="vw")
    # Hvis w ikke har 'eom' som kolonne, men som index, så reset index
    if 'eom' not in w.columns:
        if isinstance(w.index, pd.DatetimeIndex):
            w = w.reset_index()
    # Fjern kolonnen 'w'
    if 'w' in w.columns:
        w = w.drop(columns=['w'])
    
    # Sørg for, at w_opt er en DataFrame og at 'eom' er en kolonne
    if not isinstance(w_opt, pd.DataFrame):
        w_opt = pd.DataFrame(w_opt)
    if 'eom' not in w_opt.columns:
        if isinstance(w_opt.index, pd.DatetimeIndex):
            w_opt = w_opt.reset_index()
    
    # Første merge: tilføj w_opt til w på ['id', 'eom']
    w = pd.merge(w, w_opt, on=['id', 'eom'], how='left')
    
    # Merge med dates_df for at tilføje 'eom_prev'
    w = pd.merge(w, dates_df, on='eom', how='left')
    
    # Tilføj 'tr_ld1' fra data til w_opt (merge på ['id', 'eom'])
    data_subset = data[['id', 'eom', 'tr_ld1']]
    w_opt = pd.merge(w_opt, data_subset, on=['id', 'eom'], how='left')
    
    # Tilføj 'mu_ld1' fra wealth til w_opt (merge på 'eom')
    wealth_subset = wealth[['eom', 'mu_ld1']]
    w_opt = pd.merge(w_opt, wealth_subset, on='eom', how='left')
    
    # Afkastjustering: opdater w_opt['w'] = w*(1+tr_ld1)/(1+mu_ld1)
    w_opt['w'] = w_opt['w'] * (1 + w_opt['tr_ld1']) / (1 + w_opt['mu_ld1'])
    
    # Omdøb kolonner: 'w' -> 'w_prev' og 'eom' -> 'eom_prev'
    w_opt = w_opt.rename(columns={'w': 'w_prev', 'eom': 'eom_prev'})
    
    # Merge w med w_opt på ['id', 'eom_prev']
    w = pd.merge(w, w_opt, left_on=['id', 'eom_prev'], right_on=['id', 'eom_prev'], how='left')
    
    # For alle rækker, hvor eom ikke er den mindste dato, sættes w_start = w_prev
    min_eom = w['eom'].min()
    w.loc[w['eom'] != min_eom, 'w_start'] = w['w_prev']
    
    # Udfyld eventuelle manglende værdier i w_start med 0
    w['w_start'] = w['w_start'].fillna(0)
    
    # Fjern de midlertidige kolonner
    w = w.drop(columns=['eom_prev', 'w_prev'])
    
    return w

In [109]:
def mv_risky_fun(data, cov_list, wealth, dates, gam, u_vec):

    # Filtrer data: valid==True og eom findes i dates, vælg relevante kolonner og sorter efter id og eom
    data_rel = data[(data['valid'] == True) & (data['eom'].isin(dates))][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']].sort_values(by=['id', 'eom'])
    
    # Opdel data_rel efter 'eom' (svarer til split(by="eom"))
    data_split = {d: group for d, group in data_rel.groupby('eom')}
    
    # Beregn optimale weights for hver dato og for hver u-værdi
    mv_opt_all_list = []
    for d in dates:
        if d not in data_split:
            continue  # spring datoer over, der ikke findes i data_split
        data_sub = data_split[d]
        ids = data_sub['id'].values
        
        # Beregn sigma_inv: brug create_cov funktionen og beregn herefter matrixinversion
        key = d.strftime('%Y-%m-%d')
        sigma = General_Functions.create_cov(cov_list[key], ids=ids)
        sigma_inv = np.linalg.inv(sigma)
        
        # Hent forventet afkast (er) og opret en ones-vektor
        er = data_sub['pred_ld1'].values
        ones = np.ones_like(er)
        
        # Beregn de hjælpende konstanter
        a = er.T @ sigma_inv @ er
        b = np.sum(sigma_inv @ er)
        c_val = np.sum(sigma_inv @ ones)
        d_val = a * c_val - b**2
        
        # For hver u i u_vec: beregn de optimale weights
        for u in u_vec:
            factor1 = (c_val * u - b) / d_val
            factor2 = (a - b * u) / d_val
            weights = factor1 * (sigma_inv @ er) + factor2 * (sigma_inv @ ones)
            
            # Opret en midlertidig DataFrame med de beregnede weights
            temp_df = data_sub[['id', 'eom']].copy()
            temp_df['u'] = u * 12  # konverter u til årligt niveau
            temp_df['w'] = weights
            mv_opt_all_list.append(temp_df)
    
    # Kombiner alle resultater til en samlet DataFrame
    mv_opt_all = pd.concat(mv_opt_all_list, ignore_index=True)
    
    # For hver unik u-værdi, beregn porteføljen ved hjælp af w_fun og pf_ts_fun
    results_list = []
    unique_us = mv_opt_all['u'].unique()
    for u_sel in unique_us:
        # Filtrer de optimale weights for den aktuelle u-værdi og fjern kolonnen 'u'
        w_opt = mv_opt_all[mv_opt_all['u'] == u_sel].copy().drop(columns=['u'])
        
        # Beregn porteføljevægtene med w_fun (forudsætter at w_fun er defineret)
        w = w_fun(data_rel, wealth, dates, w_opt)
        
        # Udled porteføljens tidsserie med pf_ts_fun og tilføj u-værdien
        pf_result = General_Functions.pf_ts_fun(data=chars, wealth=wealth, gam=pf_set['gamma_rel'], weights=w_scaled)
        pf_result['u'] = u_sel
        results_list.append(pf_result)
    
    # Sammensæt alle portefølje-tidsserier til en samlet DataFrame
    final_result = pd.concat(results_list, ignore_index=True)
    return final_result

In [110]:
market_path="./data_fifty/market_returns_test.csv"
market = Prepare_Data.load_and_filter_market_returns_test(market_path)
market

Filen er indlæst og filtreret succesfuldt.


,eom,mkt_vw_exc
0,1952-12-31,0.029113
1,1953-01-31,-0.003177
2,1953-02-28,-0.003027
3,1953-03-31,-0.014660
4,1953-04-30,-0.029152
...,...,...
836,2022-08-31,-0.038088
837,2022-09-30,-0.093270
838,2022-10-31,0.079050
839,2022-11-30,0.048429


In [111]:
# Generate volatility range (vol_range)
vol_range = np.arange(0, 0.51, 0.01)


factor_base = portfolio_choice_functions.factor_ml_implement(data = chars, wealth = wealth, dates= dates_oos, n_pfs=settings["factor_ml"]["n_pfs"]
, gam=pf_set['gamma_rel'])
# Beregn årlig volatilitet for factor base portfolio (sd(r)*sqrt(12))
factor_base_vol = factor_base['pf']['r'].std() * np.sqrt(12)

# For hver målvolatilitet, skaler vægtene og beregn porteføljens tidsserie (pf_ts_fun)
factor_ef_list = []
for vol_target in vol_range:
    scale = vol_target / factor_base_vol if factor_base_vol != 0 else 0
    # Kopier og skaler vægt-dataframen
    w_scaled = factor_base['w'].copy()
    w_scaled['w'] = w_scaled['w'] * scale
    w_scaled['w_start'] = w_scaled['w_start'] * scale
    # Kør pf_ts_fun med de skalerede vægte
    pf_result = General_Functions.pf_ts_fun(data=chars, wealth=wealth, gam=pf_set['gamma_rel'], weights=w_scaled)
    pf_result['vol_target'] = vol_target
    factor_ef_list.append(pf_result)

# Sammensæt resultaterne i en samlet DataFrame og tilføj en type-kolonne
factor_ef = pd.concat(factor_ef_list, ignore_index=True)
factor_ef['type'] = "Factor-ML"


In [112]:
factor_ef_list = []
for vol_target in vol_range:
    scale = vol_target / factor_base_vol if factor_base_vol != 0 else 0
    # Kopier og skaler vægt-dataframen
    w_scaled = factor_base['w'].copy()
    w_scaled['w'] = w_scaled['w'] * scale
    w_scaled['w_start'] = w_scaled['w_start'] * scale
    # Kør pf_ts_fun med de skalerede vægte
    pf_result = General_Functions.pf_ts_fun(data=chars, wealth=wealth, gam=pf_set['gamma_rel'], weights=w_scaled)
    pf_result['vol_target'] = vol_target
    factor_ef_list.append(pf_result)
factor_ef_list

factor_ss = factor_ef.groupby('vol_target').apply(lambda df: pd.Series({
    'n': len(df),
    'inv': df['inv'].mean(),
    'to': df['turnover'].mean(),
    'r': df['r'].mean() * 12,
    'sd': df['r'].std() * np.sqrt(12),
    'sr_gross': (df['r'].mean() / df['r'].std() * np.sqrt(12)) if df['r'].std() != 0 else np.nan,
    'tc': df['tc'].mean() * 12,
    'r_tc': ((df['r'] - df['tc']).mean() * 12),
    'sr': ((df['r'] - df['tc']).mean() / df['r'].std() * np.sqrt(12)) if df['r'].std() != 0 else np.nan,
    'obj': (df['r'].mean() - 0.5 * df['r'].var() * pf_set['gamma_rel'] - df['tc'].mean()) * 12
})).reset_index()

u_vec = np.concatenate((np.arange(-0.5, 0.5 + 0.05, 0.05), np.array([0.6, 0.75, 1, 2]))) / 12

C:\Users\andre\AppData\Local\Temp\ipykernel_21148\2503648629.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [113]:
wealth_0 = Prepare_Data.wealth_func(wealth_end=0, end=settings['split']['test_end'], market=market, risk_free=risk_free)
mv_risky_ef = mv_risky_fun(chars, barra_cov, wealth_0, dates_oos, pf_set['gamma_rel'], u_vec)
# Udregn summary-statistikker for mv_risky_ef
mv_ss = mv_risky_ef.groupby('u').apply(lambda df: pd.Series({
    'n': len(df),
    'inv': df['inv'].mean(),
    'to': df['turnover'].mean(),
    'r': df['r'].mean() * 12,
    'sd': df['r'].std() * np.sqrt(12),
    'sr_gross': (df['r'].mean() / df['r'].std() * np.sqrt(12)) if df['r'].std() != 0 else np.nan,
    'tc': df['tc'].mean() * 12,
    'r_tc': ((df['r'] - df['tc']).mean() * 12),
    'sr': ((df['r'] - df['tc']).mean() / df['r'].std() * np.sqrt(12)) if df['r'].std() != 0 else np.nan,
    'obj': (df['r'].mean() - 0.5 * df['r'].var() * pf_set['gamma_rel'] - df['tc'].mean()) * 12
})).reset_index()

C:\Users\andre\AppData\Local\Temp\ipykernel_21148\1054396638.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [119]:
bms_path = os.path.join(output_path, "bms.csv")

# Læs bms.csv
bms = pd.read_csv(bms_path)

# Konverter eom_ret til datetime
bms["eom_ret"] = pd.to_datetime(bms["eom_ret"])
bms["type"] = "Static-ML"
with open("./data_fifty/static.pkl", "rb") as f:
    static = pickle.load(f)

In [122]:
with open("./data_fifty/static.pkl", "rb") as f:
    static = pickle.load(f)